In [1]:
import IPython
from IPython.display import clear_output, HTML, display
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
import time

In [9]:
stories_md = """
## happy path
* search_dataset
  - utter_greet
"""
%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


In [11]:
domain_yml = """
intents:
- search_dataset

actions:
- utter_greet

templates:
  utter_greet:
  - text: "Hey! How are you?"
"""
%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


In [12]:
!python -m rasa_core.train -d domain.yml -s stories.md -o models/dialogue

Processed Story Blocks: 100%|█████| 1/1 [00:00<00:00, 3120.76it/s, # trackers=3]
2019-01-30 22:34:22.987274: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 9)              0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
Total params: 5,640
Trainable params: 5,640
Non-trainable params: 0
_________________________________________________________________
2019-01-30 22:34:

In [ ]:
interpreter = RasaNLUInterpreter('models/current/nlu')
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
agent = Agent.load('models/dialogue', interpreter=interpreter)

def chatlogs_html(messages):
    messages_html = "".join(["<p>{}</p>".format(m) for m in messages])
    chatbot_html = """<div class="chat-window" {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for r in responses:
        messages.append(r.get("text"))